# Linear Regression

This notebook builds a simple linear regression model.
- Derives the MLE solution for estimating weights under Gaussian noise
- Implements Bayesian inference with Gaussian priors
- Visualizes predictive uncertainty from the posterior
- Animates gradient descent optimization


## Setup

In [2]:
import numpy as np

In [ ]:
def gaussian_pdf(x, mu, sigma):
    """
    Gaussian probability density function

    Args:
        x: point(s) to get the probability of generating from a normal distribution
        mu: mean of the normal distribution
        sigma: standard deviation of the normal distribution

    Returns:
        The probability density at x
    """
    coef = 1 / (np.sqrt(2 * np.pi * sigma ** 2))
    exponent = -((x - mu) ** 2 / (2 * sigma ** 2))
    return coef * np.exp(exponent)

np.float64(0.3520653267642995)

## MLE for Linear Regression with Gaussian Noise

We're modeling outputs of $y \in \mathbb{R}^n$ as noisy linear combinations of inputs $X \in \mathbb{R}^{n \times d}$:

$$y = X w + \epsilon$$

Where:
- $w \in \mathbb{R}^d$ as the weights to estimate
- $\epsilon \sim \mathbb{N}(0, \sigma^2 I)$ is i.i.d. Gaussian noise

Conditioned on $X$, the outputs are:

$$y \sim \mathbb{N}(X w, \sigma^2 I)$$